<a href="https://colab.research.google.com/github/valeromora/TAM_2025-1/blob/main/Final%20Proyect/Final_proyect_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set up

In [1]:
# Install necessary libraries if they are not already installed
!pip install PyDrive

In [2]:
!pip install nltk

In [3]:
!pip install -U sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile
import os
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Database

In [5]:
# https://drive.google.com/file/d/1x6HASgMM9WqaglFR-zH2_LU6eRs4l_tq/view?usp=sharing

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download the file
file_id = '1x6HASgMM9WqaglFR-zH2_LU6eRs4l_tq'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('downloaded_file.zip')

# Unzip the file
with zipfile.ZipFile('downloaded_file.zip', 'r') as zip_ref:
  zip_ref.extractall('/content/')

# List the extracted files to confirm
!ls /content/

'12EE54A_6.2.1 - Safely managed sanitation & hand-washing'
'1548EA3_6.1.1 - Safely managed drinking water'
'16BBF41_3.4.2 - Suicide'
'1772666_3.1.2 - Births attended by skilled health personnel'
'1F96863_3.4.1 - cardiovascular disease, cancer, diabetes or chronic respiratory disease'
'217795A_3.C.1 - Health worker density and distribution'
'2322814_3.2.1 - Under-five mortality rate'
'2D6FBE4_3.3.5 - Neglected tropical diseases'
'361734E_16.1.1 - Intentional homicide'
'442CEA8_3.3.3 - Malaria'
'45CA7C8_3.C.1 - Health worker density and distribution'
'5C8435F_3.C.1 - Health worker density and distribution'
'5F8A486_2.2.1 - Stunting'
 608DE39
'6A64C9A_7.1.2 - Clean fuels'
'75DDA77_3.A.1 - Age-standardized prevalence of tobacco use'
'77D059C_3.3.1 - HIV infections'
'8074BD9_3.7.1 - Women satisfied with modern methods'
'84FD3DE_3.9.3 - Unintentional poisoning'
'A37BDD6_6.3.1 - Safely treated wastewater'
'A4C49D3_3.2.2- Neonatal mortality rate'
'AC597B1_3.1.1 - Maternal mortality ratio'
'B9C

# Preparación archivos
### Extracción y enriquecimiento de información por carpeta
Este bloque de código recorre cada carpeta del dataset, busca archivos relevantes (`Metadata`, `Dictionary`, `Code list`, `Dataset`) y construye documentos enriquecidos en texto plano.

- Se extraen definiciones de variables desde el archivo **Data Dictionary**.
- Se interpretan valores codificados usando el **Code List** (si está presente).
- Se incluye el contexto general desde el archivo **Metadata**.
- Cada fila del Dataset es enriquecida con descripciones legibles para facilitar su comprensión y futura vectorización.
- El resultado se almacena en una lista de diccionarios llamada `documents`, lista para dividirse en _chunks_.

Esto permite aprovechar mejor los datos, incluso si vienen organizados de manera heterogénea.


In [6]:
base_path = "/content"
csv_count = 0
folders_with_csv = {}

for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)
    if os.path.isdir(folder_path):
        csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]
        csv_count += len(csv_files)
        folders_with_csv[folder] = csv_files

print(f"Total de archivos .csv encontrados: {csv_count}")
print(f"Resumen por carpeta:")
for folder, files in folders_with_csv.items():
    print(f"{folder}: {len(files)} archivos")


Total de archivos .csv encontrados: 139
Resumen por carpeta:
.config: 0 archivos
1772666_3.1.2 - Births attended by skilled health personnel: 3 archivos
F810947_11.6.2 - Fine particulate matter: 4 archivos
AC597B1_3.1.1 - Maternal mortality ratio: 3 archivos
84FD3DE_3.9.3 - Unintentional poisoning: 4 archivos
BEFA58B: 4 archivos
F513188_3.3.4 - Hepatitis B: 3 archivos
B9C6C79_1.a.2 - Government spending essential services: 3 archivos
12EE54A_6.2.1 - Safely managed sanitation & hand-washing: 4 archivos
2322814_3.2.1 - Under-five mortality rate: 4 archivos
361734E_16.1.1 - Intentional homicide: 4 archivos
D1223E8_6.2.1 - Safely managed sanitation & hand-washing: 4 archivos
6A64C9A_7.1.2 - Clean fuels: 4 archivos
EF93DDB: 4 archivos
5F8A486_2.2.1 - Stunting: 3 archivos
C288D13_3.3.2 - Tuberculosis: 3 archivos
A37BDD6_6.3.1 - Safely treated wastewater: 3 archivos
A4C49D3_3.2.2- Neonatal mortality rate: 4 archivos
F8524F2_5.2.1 - Intimate partner violence (last 12 months): 3 archivos
D6176E

In [7]:
# Ruta base donde se encuentran las carpetas con los datasets y archivos relacionados
base_path = "/content"

# Lista que almacenará los documentos generados por cada carpeta,
# donde cada documento es un texto enriquecido con metadata, definiciones y datos
documents = []

# Recorremos cada carpeta dentro de la ruta base
for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)
    # Saltamos si no es carpeta (p.ej., archivos sueltos)
    if not os.path.isdir(folder_path):
        continue

    # Diccionarios de mapeo para traducir y dar contexto:
    # data_dict_map: mapea nombre de variable → definición textual
    data_dict_map = {}
    # code_list_map: mapea (dimensión, código) → descripción del código
    code_list_map = {}

    # Variable para acumular el texto de metadata (solo un archivo por carpeta)
    metadata_text = ""

    # Búsqueda y extracción del archivo de Metadata para la carpeta actual
    # Solo se procesa el primer archivo que cumpla la condición
    for filename in os.listdir(folder_path):
        if "Metadata" in filename and filename.endswith(".csv"):
            try:
                df_meta = pd.read_csv(os.path.join(folder_path, filename))
                metadata_text += f"📄 Metadata de {filename}:\n"
                # Recorremos las filas sin índice para extraer clave y valor
                for row in df_meta.itertuples(index=False):
                    if len(row) >= 2:
                        key = str(row[0]).strip()
                        value = str(row[1]).strip()
                        metadata_text += f"{key}: {value}\n"
            except Exception as e:
                print(f"No se pudo leer metadata en {folder}: {e}")
            break  # Solo procesamos un archivo Metadata por carpeta

    # Primera pasada: cargamos diccionarios para darle sentido a los datos
    for filename in os.listdir(folder_path):
        # Procesamos solo archivos CSV
        if not filename.endswith(".csv"):
            continue

        file_path = os.path.join(folder_path, filename)
        try:
            df = pd.read_csv(file_path)
        except Exception as e:
            print(f"No se pudo leer {filename}: {e}")
            continue

        # Si es un archivo Data Dictionary, cargamos definiciones de variables
        if "Dictionary" in filename or "Data Dictionary" in filename:
            for row in df.itertuples(index=False):
                if len(row) >= 2:
                    var = str(row[0]).strip()
                    definition = str(row[1]).strip()
                    # Guardamos mapeo: nombre_variable -> definición
                    data_dict_map[var] = definition

        # Si es un archivo Code List, cargamos descripciones de códigos por dimensión
        elif "Code list" in filename:
            for row in df.itertuples(index=False):
                if len(row) >= 4:
                    dim = str(row[0]).strip()     # Dimensión o categoría
                    key = str(row[1]).strip()     # Código o clave
                    name = str(row[2]).strip()    # Nombre o etiqueta
                    desc = str(row[3]).strip()    # Descripción detallada
                    # Guardamos mapeo: (dimensión, código) -> descripción enriquecida
                    code_list_map[(dim, key)] = f"{name}: {desc}"

    # Segunda pasada: procesamos archivos que contengan los datos en sí
    for filename in os.listdir(folder_path):
        # Saltamos si no es CSV o si es Dictionary o Code list (ya procesados)
        if not filename.endswith(".csv") or "Dictionary" in filename or "Code list" in filename:
            continue

        file_path = os.path.join(folder_path, filename)
        try:
            df = pd.read_csv(file_path)
        except Exception as e:
            print(f"No se pudo leer {filename}: {e}")
            continue

        text_content = ""

        # Si por algún motivo volvemos a encontrar metadata, la agregamos
        if "Metadata" in filename:
            text_content += f"🧾 Metadata extraída de {filename}:\n"
            for row in df.itertuples(index=False):
                if len(row) >= 2:
                    key = str(row[0]).strip()
                    value = str(row[1]).strip()
                    text_content += f"{key}: {value}\n"

        # Si es un dataset con datos, construimos textos enriquecidos
        elif "Dataset" in filename:
            text_content += f"📊 Datos enriquecidos de {filename}:\n"
            # Recorremos fila por fila
            for row in df.itertuples(index=False):
                row_description = []
                # Para cada columna, buscamos su definición y posible descripción de valor
                for col in df.columns:
                    # Valor en la celda actual
                    value = getattr(row, col, "")
                    # Nombre legible para la variable, si existe
                    label = data_dict_map.get(col, col)

                    # Si el valor es una cadena y existe descripción en code_list para esa dimensión y valor
                    if isinstance(value, str) and (col, value) in code_list_map:
                        value_desc = code_list_map[(col, value)]
                        row_description.append(f"{label}: {value_desc}")
                    else:
                        # Si no hay descripción especial, usamos el valor tal cual
                        row_description.append(f"{label}: {value}")

                # Concatenamos la descripción enriquecida para la fila completa
                text_content += " | ".join(row_description) + "\n"

        else:
            # Archivos no reconocidos se reportan para control
            print(f"Archivo sin tipo conocido: {filename}")
            continue

        # Finalmente, agregamos a la lista de documentos con su fuente y contenido enriquecido
        documents.append({
            "source": f"{folder}/{filename}",
            "content": metadata_text + "\n" + text_content
        })

Archivo sin tipo conocido: mnist_test.csv
Archivo sin tipo conocido: california_housing_train.csv
Archivo sin tipo conocido: california_housing_test.csv
Archivo sin tipo conocido: mnist_train_small.csv


In [8]:
print(f"Total documentos extraídos: {len(documents)}")
print(documents[0]['source'])
print(documents[0]['content'][:1000])  # muestra los primeros caracteres del primer documento

Total documentos extraídos: 76
1772666_3.1.2 - Births attended by skilled health personnel/170_1772666_Metadata_2025-04-07.csv
📄 Metadata de 170_1772666_Metadata_2025-04-07.csv:
Name: Proportion of births attended by skilled health personnel (%)
Short name: Proportion of births attended by skilled health personnel (%)
Indicator unique identifier: 1772666
Indicator codes: MDG_0000000025
Also known as: SDG indicator 3.1.2
SDG Goal: 3.1.2 – Births attended by skilled health personnel
Short description: Proportion of births attended by skilled health personnel.
(SDG 3.1.2)
Definition: Proportion of births attended by skilled health personnel (generally doctors, nurses or midwives but can refer to other health professionals providing childbirth care) is the proportion of childbirths attended by professional health personnel.
According to the current definition (1) these are competent maternal and newborn health (MNH) professionals educated, trained and regulated to national and internationa

# Chunking

Se busca dividir cada content del documento en trozos más pequeños.
* Divide el texto en grupos de 500 palabras

* Repite las últimas 50 palabras del chunk anterior (para mantener contexto)

In [9]:
MAX_CHUNK_WORDS = 500
OVERLAP = 50

chunks = []

for doc in documents:
    source = doc["source"]
    words = doc["content"].split()

    for i in range(0, len(words), MAX_CHUNK_WORDS - OVERLAP):
        chunk_words = words[i:i + MAX_CHUNK_WORDS]
        chunk_text = ' '.join(chunk_words)

        chunks.append({
            "source": source,
            "content": chunk_text
        })

In [10]:
print(f"Total de chunks: {len(chunks)}")
print("Ejemplo de chunk:\n", chunks[0]["content"][:700])

Total de chunks: 576
Ejemplo de chunk:
 📄 Metadata de 170_1772666_Metadata_2025-04-07.csv: Name: Proportion of births attended by skilled health personnel (%) Short name: Proportion of births attended by skilled health personnel (%) Indicator unique identifier: 1772666 Indicator codes: MDG_0000000025 Also known as: SDG indicator 3.1.2 SDG Goal: 3.1.2 – Births attended by skilled health personnel Short description: Proportion of births attended by skilled health personnel. (SDG 3.1.2) Definition: Proportion of births attended by skilled health personnel (generally doctors, nurses or midwives but can refer to other health professionals providing childbirth care) is the proportion of childbirths attended by professional health person


# Vectorización (embeddings)
Los modelos de lenguaje como los usados en RAG no buscan respuestas de forma exacta por coincidencia de palabras, sino que comparan significados. Para hacer esto, necesitamos representar cada fragmento de texto (chunk) como un vector en un espacio multidimensional. Luego podemos encontrar los textos más relevantes comparando su distancia semántica.

Utilizamos el modelo all-MiniLM-L6-v2 de la librería sentence-transformers, el cual transforma cada chunk de texto en un vector numérico de 384 dimensiones que resume su contenido de forma semántica. Luego extraemos los textos de los chunks previamente generados y los vectorizamos

In [11]:
# Cargar el modelo
model = SentenceTransformer('all-MiniLM-L6-v2')

# Extraer solo el texto de los chunks
texts = [chunk['content'] for chunk in chunks]

# Vectorizar los textos
embeddings = model.encode(texts, show_progress_bar=True)

# Convertir a matriz numpy
embeddings_np = np.array(embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

Usamos FAISS (Facebook AI Similarity Search) para almacenar los vectores y poder hacer búsquedas por similitud. Esto nos permite recuperar los chunks más cercanos a una consulta (query) en términos de significado.

In [12]:
# Creamos un índice FAISS para búsquedas por similitud
dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)

# Añadir los vectores
index.add(embeddings_np)

In [13]:
# Guardar los metadatos (puede ser útil para devolver respuestas)
chunk_sources = [chunk["source"] for chunk in chunks]

Con el índice creado, ahora podemos consultar el sistema en lenguaje natural. FAISS devuelve los chunks más relevantes con base en la similitud semántica entre la consulta y los documentos vectorizados.

In [14]:
# Ejemplo de búsqueda
query = "Distribución por edad y género de la población en 2018"
query_vec = model.encode([query])

# Buscar los 5 chunks más similares
distances, indices = index.search(query_vec, k=5)

# Mostrar los resultados
for i in indices[0]:
    print("🔹 Source:", chunks[i]["source"])
    print(chunks[i]["content"][:300])
    print("-" * 80)

🔹 Source: AC597B1_3.1.1 - Maternal mortality ratio/170_AC597B1_Metadata_2025-04-07.csv
April 2025 Next expected data update: 2026-2028 Expected frequency of data release: Every 3 - 5 years Publishing institution: WHO_HQ_UHL_SRH_MPH Data type: Float Unit of measure: Number of maternal deaths per 100 000 live births Value display label: deaths per 100 000 live births Short value display
--------------------------------------------------------------------------------
🔹 Source: 217795A_3.C.1 - Health worker density and distribution/170_217795A_Metadata_2024-05-20.csv
(revision 2008), the International Standard Classification of Education (revision 2011) and the International Standard Industrial Classification of All Economic Activities (revision 4). Direction of progress: Figures increasing over time demonstrates progress. Indicator status: Actively collected an
--------------------------------------------------------------------------------
🔹 Source: 1772666_3.1.2 - Births attended by ski